In [1]:
import pandas as pd
import math
from statistics import mode 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [4]:
df=pd.read_csv("C:\\Users\\Trivikram\\ML_Projects\\Data\\adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','capital gain','capital loss','hours per week','native country','Salary'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
def removeQuestionMark(inpCol,X):
    for i in inpCol:
        repl=X[i].value_counts().keys().tolist()[0]
        X[i]=X[i].replace(to_replace=' ?',value=repl)
    return X

In [6]:
y=df[['Salary']]
X=df[['Age','fnlwgt','education num','capital gain','capital loss','hours per week','Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
inpCol=list(X.columns)
outCol=list(y.columns)
X=removeQuestionMark(inpCol,X)
categCol=['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']
contCol=['Age','fnlwgt','education num','capital gain','capital loss','hours per week']
X

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Age,fnlwgt,education num,capital gain,capital loss,hours per week,Workplace,education,race,sex,marital-stauts,occupation,relationship,native country
0,39,77516,13,2174,0,40,State-gov,Bachelors,White,Male,Never-married,Adm-clerical,Not-in-family,United-States
1,50,83311,13,0,0,13,Self-emp-not-inc,Bachelors,White,Male,Married-civ-spouse,Exec-managerial,Husband,United-States
2,38,215646,9,0,0,40,Private,HS-grad,White,Male,Divorced,Handlers-cleaners,Not-in-family,United-States
3,53,234721,7,0,0,40,Private,11th,Black,Male,Married-civ-spouse,Handlers-cleaners,Husband,United-States
4,28,338409,13,0,0,40,Private,Bachelors,Black,Female,Married-civ-spouse,Prof-specialty,Wife,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,Private,Assoc-acdm,White,Female,Married-civ-spouse,Tech-support,Wife,United-States
32557,40,154374,9,0,0,40,Private,HS-grad,White,Male,Married-civ-spouse,Machine-op-inspct,Husband,United-States
32558,58,151910,9,0,0,40,Private,HS-grad,White,Female,Widowed,Adm-clerical,Unmarried,United-States
32559,22,201490,9,0,0,20,Private,HS-grad,White,Male,Never-married,Adm-clerical,Own-child,United-States


In [7]:
enc = OneHotEncoder(handle_unknown='ignore')
binaryValues=enc.fit_transform(X[categCol]).toarray()
newCol=list(enc.get_feature_names(categCol))
categData = pd.DataFrame(binaryValues,columns=newCol,index=X.index)
X=X[contCol].merge(categData,left_index=True,right_index=True)
scaler = StandardScaler()
X=pd.DataFrame(data=scaler.fit_transform(X),columns=X.columns)
df=X

In [8]:
X=df[contCol]

In [10]:
neight = KNeighborsClassifier()
parameters={'n_neighbors':([i for i in range(2,20)])}
clf = GridSearchCV(neight, parameters)

In [19]:
pipe_kn = Pipeline([('KNN', neight)])
pipe_gnb = Pipeline([('NB', GaussianNB())])
pipelines = [pipe_kn, pipe_gnb]
pipe_dict = {0: 'K Nearest Neighbour', 1: 'Gausscian Naive Bayes'}
for pipe in pipelines:
    pipe.fit(X, y)
for idx, val in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X, y)))

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


K Nearest Neighbour pipeline test accuracy: 0.859
Gausscian Naive Bayes pipeline test accuracy: 0.796


In [12]:
# Create a pipeline
pipe = Pipeline(steps=[('scl',None),('classifier',GaussianNB())])

# Create space of candidate learning algorithms and their hyperparameters
search_space = [{'classifier':[KNeighborsClassifier()],'scl':[ StandardScaler()],
                'classifier__n_neighbors':range(2,20)}]


In [13]:
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0)

In [14]:
clf.fit(X,y)

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\pipeline.py:356: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scl', None),
                                       ('classifier',
                                        GaussianNB(priors=None,
                                                   var_smoothing=1e-09))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'classifier': [KNeighborsClassifier(algorithm='auto',
                                                              leaf_size=30,
                                                              metric='minkowski',
                                                              metric_params=None,
                                                              n_jobs=None,
                                                              n_neighbors=16,
                                                              p=2,
                              

In [67]:
clf.score(X_test,y_test)

0.8361142389190296

In [70]:
clf.best_estimator_

Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=16, p=2,
                                      weights='uniform'))],
         verbose=False)

In [16]:
neight = KNeighborsClassifier()
pipe_kn = Pipeline([('KNN', neight)])

In [30]:
pipe_lr = Pipeline([('scl', StandardScaler()),('knn', KNeighborsClassifier())],[('NB', GaussianNB())])

In [31]:
pipe_lr

Pipeline(memory=[('NB', GaussianNB(priors=None, var_smoothing=1e-09))],
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

C:\Users\Trivikram\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
